<a href="https://colab.research.google.com/github/Esther-Mbanzabigwi/summative_Mental_chatbot/blob/main/Mental_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Preprocessing**

In [1]:
import json
import pandas as pd
import re
import string
import nltk

# Download necessary NLTK data for tokenization
nltk.download("punkt")

# If using opendatasets to download from Kaggle (ensure proper Kaggle API setup)
!pip install opendatasets

import opendatasets as od
od.download("https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data")


# Load JSON data
with open("/content/mental-health-conversational-data/intents.json", "r") as file:
    data = json.load(file)

# Transform JSON into a list of dictionaries for each pattern
intents_data = []
for intent in data["intents"]:
    tag = intent["tag"]
    for pattern in intent["patterns"]:
        intents_data.append({"tag": tag, "pattern": pattern})

# Convert to DataFrame
df = pd.DataFrame(intents_data)

# Define a text cleaning function
def clean_text(text):
    text = text.lower()  # convert to lowercase
    text = text.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    text = re.sub(r"\s+", " ", text).strip()  # remove extra spaces
    return text

# Apply text cleaning
df["cleaned_pattern"] = df["pattern"].apply(clean_text)

# Save the preprocessed data as CSV
output_file = "/content/mental-health-conversational-data/intents.csv"
df.to_csv(output_file, index=False, encoding="utf-8")

print("Preprocessing complete! Cleaned data saved at:", output_file)
print(df.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: esthermbanzabigwi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data


100%|██████████| 11.8k/11.8k [00:00<00:00, 7.63MB/s]


Preprocessing complete! Cleaned data saved at: /content/mental-health-conversational-data/intents.csv
        tag           pattern  cleaned_pattern
0  greeting                Hi               hi
1  greeting               Hey              hey
2  greeting  Is anyone there?  is anyone there
3  greeting          Hi there         hi there
4  greeting             Hello            hello


**Model Fine-Tuning and Evaluation**

In [2]:
import json
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

# Load the CSV dataset
file_path = "/content/mental-health-conversational-data/intents.csv"
df = pd.read_csv(file_path)

# Create a label mapping
labels = sorted(df["tag"].unique().tolist())  # Get sorted list of all labels
label_map = {label: idx for idx, label in enumerate(labels)}
df["label"] = df["tag"].map(label_map)

# Display dataset info
print("Number of unique intents:", len(labels))
print("Label mapping:", label_map)
print(df["tag"].value_counts())

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Custom Dataset class
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

# Split dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["cleaned_pattern"], df["label"], test_size=0.2, random_state=42
)

train_dataset = IntentDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = IntentDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

# Load pre-trained DistilBERT for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(labels))

# Define training arguments
training_args = TrainingArguments(
    output_dir="./model_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("fine_tuned_chatbot")
tokenizer.save_pretrained("fine_tuned_chatbot")
print("\n Fine-tuning complete! Model saved in 'fine_tuned_chatbot/'")

# === FIXED EVALUATION SECTION ===
# Get predictions on validation set
predictions = trainer.predict(val_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)
true_labels = val_labels.tolist()

# Get only unique labels that appear in the validation set
unique_classes = sorted(set(true_labels) | set(predicted_labels))

#Filter target_names based on unique labels in validation set
target_names_eval = [labels[i] for i in unique_classes]

#Ensure classification report matches correct number of classes
accuracy = accuracy_score(true_labels, predicted_labels)
report = classification_report(true_labels, predicted_labels, labels=unique_classes, target_names=target_names_eval)

print(f"\n Model Accuracy: {accuracy:.4f}")
print(report)


Number of unique intents: 80
Label mapping: {'about': 0, 'afternoon': 1, 'anxious': 2, 'ask': 3, 'casual': 4, 'creation': 5, 'death': 6, 'default': 7, 'depressed': 8, 'done': 9, 'evening': 10, 'fact-1': 11, 'fact-10': 12, 'fact-11': 13, 'fact-12': 14, 'fact-13': 15, 'fact-14': 16, 'fact-15': 17, 'fact-16': 18, 'fact-17': 19, 'fact-18': 20, 'fact-19': 21, 'fact-2': 22, 'fact-20': 23, 'fact-21': 24, 'fact-22': 25, 'fact-23': 26, 'fact-24': 27, 'fact-25': 28, 'fact-26': 29, 'fact-27': 30, 'fact-28': 31, 'fact-29': 32, 'fact-3': 33, 'fact-30': 34, 'fact-31': 35, 'fact-32': 36, 'fact-5': 37, 'fact-6': 38, 'fact-7': 39, 'fact-8': 40, 'fact-9': 41, 'friends': 42, 'goodbye': 43, 'greeting': 44, 'happy': 45, 'hate-me': 46, 'hate-you': 47, 'help': 48, 'jokes': 49, 'learn-mental-health': 50, 'learn-more': 51, 'location': 52, 'meditation': 53, 'mental-health-fact': 54, 'morning': 55, 'name': 56, 'neutral-response': 57, 'night': 58, 'no-approach': 59, 'no-response': 60, 'not-talking': 61, 'pandora-

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss
1,4.397900,4.363758
2,4.309700,4.314518
3,4.081400,4.231514
4,3.990200,4.156981
5,3.818200,4.107536


Epoch,Training Loss,Validation Loss
1,4.397900,4.363758
2,4.309700,4.314518
3,4.081400,4.231514
4,3.990200,4.156981
5,3.818200,4.107536
6,3.714600,4.066431
7,3.647500,4.032698
8,3.534200,4.009271
9,3.489000,3.989314
10,3.392200,3.985734



 Fine-tuning complete! Model saved in 'fine_tuned_chatbot/'



 Model Accuracy: 0.1702
                     precision    recall  f1-score   support

              about       0.00      0.00      0.00         0
            anxious       0.00      0.00      0.00         1
             casual       0.12      0.50      0.20         2
              death       1.00      1.00      1.00         2
            default       0.00      0.00      0.00         3
          depressed       0.00      0.00      0.00         1
               done       0.00      0.00      0.00         1
            fact-19       0.00      0.00      0.00         1
            fact-22       0.00      0.00      0.00         1
            fact-29       0.00      0.00      0.00         1
             fact-3       0.00      0.00      0.00         1
             fact-5       0.00      0.00      0.00         2
             fact-6       0.00      0.00      0.00         1
             fact-9       0.00      0.00      0.00         1
            goodbye       0.00      0.00      0.00         

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

This script loads the saved model and tokenizer, defines a prediction function that not only outputs the intent but also maps it to a natural response. You can modify or expand the response dictionary as needed.

**Deployment with Streamlit**

In [3]:
!pip install streamlit
import sys
!{sys.executable} -m pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [4]:
!pip install pyngrok
from pyngrok import ngrok


In [5]:
!pip install streamlit torch transformers pandas scikit-learn pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

**Main Chatbot UI**

In [24]:

%%writefile app.py
import streamlit as st
import torch
import torch.nn.functional as F
import time
import random
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from inference import predict_intent  # Import function from inference.py

# Load the fine-tuned model
model_path = "fine_tuned_chatbot"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

# Function to add typing effect
def chatbot_typing_effect(response):
    with st.spinner("🤖 Chatbot is thinking..."):
        time.sleep(1.5)
    st.write(response)

# Apply a calming background color & UI styling
st.markdown(
    """
    <style>
    body {
        background-color: #E8F5E9;
    }
    .stButton>button {
        background-color: #81C784;
        color: white;
        font-size: 18px;
        border-radius: 10px;
        padding: 10px 20px;
    }
    .stTextInput>div>div>input {
        font-size: 16px;
        padding: 12px;
        border-radius: 10px;
        border: 2px solid #AED581;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Chatbot UI
st.image("https://cdn-icons-png.flaticon.com/512/4727/4727429.png", width=120)
st.title("💙 Your Mental Health Companion")
st.write("Hello there! I'm here to listen. You can share how you're feeling, and I'll do my best to support you. 💬")

# Soothing Background Music
st.markdown(
    """
    <audio autoplay loop>
        <source src="https://www.bensound.com/bensound-music/bensound-relaxing.mp3" type="audio/mp3">
    </audio>
    """,
    unsafe_allow_html=True
)

# Meditation Feature
st.write("💆 Need a quick relaxation break? Try a 2-minute guided meditation.")
if st.button("🎧 Start Meditation"):
    st.audio("https://www.bensound.com/bensound-music/bensound-meditation.mp3")

# User input
user_input = st.text_input("How are you feeling today?")

fallback_responses = [
    "I'm here for you. Could you tell me a bit more about how you're feeling? 💙",
    "I want to help. Can you describe what’s bothering you?",
    "It sounds like you’re going through something. Would you like some relaxation techniques?",
    "I'm listening. You can share as much or as little as you’d like. 💬"
]

if st.button("Submit"):
    if user_input.strip():
        predicted_intent, confidence, response = predict_intent(user_input, model, tokenizer, device)

        if confidence < 0.3:
            chatbot_typing_effect(random.choice(fallback_responses))
        else:
            st.write(f"**🧠 Predicted Intent:** {predicted_intent} (Confidence: {confidence:.2f})")

            if predicted_intent == "stress":
                chatbot_typing_effect("🌿 Take a deep breath. You’re not alone. Would you like some stress management tips?")
            elif predicted_intent == "anxiety":
                chatbot_typing_effect("💜 It’s okay to feel anxious sometimes. Would you like to try a grounding technique?")
            elif predicted_intent == "depression":
                chatbot_typing_effect("❤️ I'm really sorry you’re feeling this way. You're not alone, and I'm here to help.")
            else:
                chatbot_typing_effect(response)
    else:
        st.write("⚠️ Please enter a valid message.")





Writing app.py


**Model Inference for Intent Detection**

In [7]:
%%writefile inference.py
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the fine-tuned model and tokenizer
model_path = "fine_tuned_chatbot"

# Define response dictionary
intent_responses = {
    "greeting": "Hello! How can I help you today?",
    "stress": "I'm sorry you're feeling stressed. Would you like some tips on relaxation?",
    "anxiety": "It sounds like you're experiencing anxiety. I can suggest breathing exercises.",
    "depression": "I'm really sorry you're feeling depressed. Talking to someone may help.",
    "relaxation": "Try deep breathing, meditation, or a short walk to relax.",
    "other": "Could you provide more details? I'm here to help."
}

# Function for intent prediction
def predict_intent(text, model, tokenizer, device, temperature=0.7):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits / temperature
    probabilities = F.softmax(logits, dim=1)
    predicted_idx = torch.argmax(probabilities, dim=1).item()
    confidence = probabilities[0][predicted_idx].item()

    # Get labels dynamically
    num_labels = model.config.num_labels
    labels = [f"intent_{i}" for i in range(num_labels)]  # Placeholder labels

    # Ensure valid labels
    predicted_intent = labels[predicted_idx] if predicted_idx < len(labels) else "other"
    response = intent_responses.get(predicted_intent, "take a deep breath first.")

    return predicted_intent, confidence, response




Writing inference.py


**Requirements File**

In [8]:
%%writefile requirements.txt
streamlit
torch
transformers
pandas
scikit-learn



Writing requirements.txt


In [9]:
!pip install streamlit transformers torch ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.1 MB/s eta 0:00:00


In [14]:
!pkill streamlit


In [15]:
!streamlit run app.py &>/dev/null &


In [16]:
!pip install pyngrok
!ngrok authtoken 2tUyqyKZJGxI4hd1UqChpxZw29P_2cpasSTxfhak2G2M4qYcg


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
from pyngrok import ngrok

# Explicitly define an HTTP tunnel
public_url = ngrok.connect("8501", "http")
print(f"🚀 Your chatbot is live at: {public_url}")


🚀 Your chatbot is live at: NgrokTunnel: "https://170c-34-132-69-254.ngrok-free.app" -> "http://localhost:8501"
